In [1]:
from memory import Memory
from classifier_agent import ClassifierAgent
from json_agent import JSONAgent
from email_agent import EmailAgent

mem = Memory(db_path=":memory:")

In [2]:
import os

def run_pipeline(file_path):
    classifier = ClassifierAgent(mem)
    fmt, intent, payload = classifier.classify(file_path)
    print(f"► File: {os.path.basename(file_path)}")
    print(f"  • Detected format: {fmt}")
    print(f"  • Detected intent: {intent}")
    
    if fmt == "JSON":
        agent = JSONAgent(mem)
        extracted, anomalies = agent.process(payload, intent, file_path)
        print("  → JSON Agent output:")
        print("    • Extracted fields:", extracted)
        if anomalies:
            print("    • Anomalies:", anomalies)
    elif fmt == "Email":
        agent = EmailAgent(mem)
        crm = agent.process(payload, file_path)
        print("  → Email Agent output:")
        for k,v in crm.items():
            print(f"    • {k}: {v}")
    else:
        snippet = payload[:200].replace("\n"," ")
        print("  → PDF text snippet:", snippet, "…")
    print()

In [3]:
samples = ["sample_invoice.pdf", "sample_input.json", "sample_email.txt"]
for f in samples:
    run_pipeline(f)

► File: sample_invoice.pdf
  • Detected format: PDF
  • Detected intent: Invoice
  → PDF text snippet: INVOICE Invoice Number: INV-001 Date: 2025-05-28 Bill To: Acme Corporation 123 Elm Street Cityville, CV 12345 Description Qty Unit Price Total Widget A 5 $9.99 $49.95 Gadget B 2 $19.99 $39.98 Service  …

► File: sample_input.json
  • Detected format: JSON
  • Detected intent: Invoice
  → JSON Agent output:
    • Extracted fields: {'invoice_number': 'INV-002', 'date': '2025-05-29', 'customer': {'name': 'Beta Corp', 'address': '123 Market St'}, 'items': [{'description': 'Widget', 'quantity': 10, 'unit_price': 9.99}, {'description': 'Gadget', 'quantity': 3, 'unit_price': 19.99}]}

► File: sample_email.txt
  • Detected format: Email
  • Detected intent: Complaint
  → Email Agent output:
    • sender: alice@example.com
    • subject: Complaint: Defective product received
    • body: Hello,
I would like to file a complaint regarding the product I received last week. It is defective...
Pleas

In [4]:
import pandas as pd

rows = mem.fetch_all()
df = pd.DataFrame(rows, columns=["id","source","format","intent","timestamp","extracted","thread_id"])
df

C:\Users\abhis\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\abhis\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


,id,source,format,intent,timestamp,extracted,thread_id
0,1,sample_invoice.pdf,PDF,Invoice,2025-05-28T18:52:33.234549,,sample_invoice.pdf
1,2,sample_input.json,JSON,Invoice,2025-05-28T18:52:33.236578,,sample_input.json
2,3,sample_input.json,JSON,Invoice,2025-05-28T18:52:33.237118,"{'invoice_number': 'INV-002', 'date': '2025-05...",sample_input.json
3,4,sample_email.txt,Email,Complaint,2025-05-28T18:52:33.237623,,sample_email.txt
4,5,sample_email.txt,Email,Complaint,2025-05-28T18:52:33.238142,"{'sender': 'alice@example.com', 'subject': 'Co...",sample_email.txt
